In [468]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl import load_workbook
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [469]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

def check_unique(x):
    # Функция для нахождения разночтений в значениях
    temp_lst = x.split(';')
    temp_set = set(temp_lst)
    if 'nan' in temp_set:
        return 'Не заполнены значения площади лесотаксационного выдела!!!'
    else:
        return 'Все в порядке' if len(temp_set) == 1 else 'Ошибка!!!'

def main_check_unique(x):
    # Функция для проверки корректности заполнения площади выдела
    temp_str = ';'.join(x) # Склеиваем все значения
    temp_lst = temp_str.split(';') # Создаем список разбивая по ;
    temp_set = set(temp_lst) # Превращаем в множество
    if len(temp_set) > 1: # Если длина множества больше 1 то есть погрешности
        return 0
    elif'nan' in temp_set:# если есть нан то не заполнены площади выдела
        return 0
    else:# Если все в порядке то возвращаем единственный элемент списка
        return convert_to_float(temp_lst[0])

def convert_to_float(x):
    """
    Функция для конвертирования строки в флоат.
    :param x: строка
    :return:
    """
    # очищаем от пробелов и возможных заполнения ячейки пробелом
    
    try:
        return float(x)
    except ValueError:
        return 0
def convert_to_int_transfer(cell):
    """
    Функция для конвертации в int
    """
    try:
        return int(cell)
    except ValueError:
        return 0
    
def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return ''

In [470]:
group_rb_region_transfer_3_to_reestr = 0

file_transfer_to_upp = 'data/Сравнение таблиц и присвоение данных (площади)/Таблица 3 с дополнительными колонками..xlsx'
file_transfer_reestr = 'data/Сравнение таблиц и присвоение данных (площади)/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'

# file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Якутия октябрь_Таблица 3с доп. колонками.xlsx'
# file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Якутия 2022-11-27_72_Реестр УПП по новой форме.xlsx'

path_to_end_folder = 'data/'
header_table3 = 'data/Сравнение таблиц и присвоение данных (площади)/Заголовок Таблица 3 от 08.11.xlsx'

In [471]:
# Создаем список колонок которые нужно загрузить
use_cols_upp = list(range(6))
use_cols_table3 = list(range(34))

# Загружаем датафреймы
df = pd.read_excel(file_transfer_reestr,skiprows=8,usecols=use_cols_upp)# Реестр УПП
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=use_cols_table3) # Таблица 3

In [472]:
df_table_3.head()

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,13.0,...,0,0,0,0,0,1,201,NaN,0.0,2015
1,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,300.0,...,0,0,0,0,0,1,201,NaN,0.0,2020
2,2022-06-29,NaN,0.3,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,500.0,...,0,0,0,0,0,1,201,NaN,0.0,2021
3,2022-06-29,NaN,2.3,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,400.0,...,более 40%,более 40%,2022-04-05 00:00:00,1,0,1,103,NaN,2.3,2015
4,2022-06-29,NaN,0.8,Республика Бурятия,Улан-Удэнское,Березовское,NaN,39,8,2.0,...,более 40%,более 40%,2022-04-05 00:00:00,1,0,1,103,NaN,0.8,2015


In [473]:
df.head()

,1,2,3,4,5,6
0,Ангоянское,Ангоянское,NaN,32,1,13
1,Ангоянское,Ангоянское,NaN,32,2,7
2,Ангоянское,Ангоянское,NaN,32,3,11
3,Ангоянское,Ангоянское,NaN,32,5,10
4,Ангоянское,Ангоянское,NaN,32,7,78


In [474]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df.columns = list(map(str,list(df.columns)))
# на всякий случай очищаем от пробельных символов
df.columns = list(map(lambda x:x.replace(" ",""),list(df.columns)))

In [475]:
"""
Соответствие названий колонок используемым в программе номерам колонок
Лесничество -1
Участковое лесничество- 2
Урочище - 3
Номер лесного квартала -4
Номер лесотаксационного выдела -5
Площадь лесотаксационного выдела, га - 6
"""

'\nСоответствие названий колонок используемым в программе номерам колонок\nЛесничество -1\nУчастковое лесничество- 2\nУрочище - 3\nНомер лесного квартала -4\nНомер лесотаксационного выдела -5\nПлощадь лесотаксационного выдела, га - 6\n'

In [476]:
df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',
                  '6':'Площадь лесотаксационного выдела, га'},inplace=True)



In [477]:
# Заполняем Nan чтобы группировка прошла успешно
df['Урочище'] = df['Урочище'].fillna('Название урочища не заполнено')

In [478]:
df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
    lambda x: x.replace(',', '.'))

# Группируем
df_upp = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                       'Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                               })

df_upp=df_upp.astype(str)


In [479]:
# Извлекаем индексы в колонки
df_upp = df_upp.reset_index()

In [480]:
df_upp.head()


,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га"
0,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13.0
1,Ангоянское,Ангоянское,Название урочища не заполнено,32,2,7.0
2,Ангоянское,Ангоянское,Название урочища не заполнено,32,3,11.0
3,Ангоянское,Ангоянское,Название урочища не заполнено,32,5,10.0
4,Ангоянское,Ангоянское,Название урочища не заполнено,32,7,78.0


In [481]:
# Обрабатываем колонки таблицы 3
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

In [482]:
# делаем строковыми первые 5 колонок в датафрейме Реестр УПП
df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']] = df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']].astype(str)
df_upp['ID'] = df_upp.loc[:,['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела'] ].sum(axis=1)
df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace(' ',''))
df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace('nan',''))

In [483]:
# Заменяем nan в урочище 
df_table_3['7'] = df_table_3['7'].fillna('Название урочища не заполнено')

In [484]:
# делаем строковыми колонки по которым будем соединять в датафрейме 3 таблицы
df_table_3[['5','6','7','8','9']] = df_table_3[['5','6','7','8','9']].astype(str)
df_table_3['ID'] = df_table_3.loc[:,['5','6','7','8','9'] ].sum(axis=1)
df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace(' ',''))
df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace('nan',''))

In [518]:
# Мержим по полю айди
merged_df = pd.merge(df_upp,df_table_3,how='outer',left_on='ID',right_on='ID',indicator=True)

In [519]:
merged_df['Площадь'] = np.where(merged_df['_merge'] == 'both',merged_df['Площадь лесотаксационного выдела, га'],merged_df['10'])

In [520]:
# Приводим к инту колонки чтобы отсортировать их
merged_df['8'] = merged_df['8'].apply(convert_to_int_transfer)
merged_df['9'] = merged_df['9'].apply(convert_to_int_transfer)

In [522]:
merged_df.to_excel('Тест.xlsx',index=False)

In [488]:
merged_df.sort_values(['5','6','7','8','9'],inplace=True)

# Присваиваем колонке 10 значение колонки созданной колонки 10
merged_df['10'] = merged_df['Площадь']

In [489]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Нет в таблице 3'
wb.create_sheet(title='Нет в реестре УПП', index=1)
wb.create_sheet(title='Выделы с обновленной площадью', index=2)

<Worksheet "Выделы с обновленной площадью">

In [490]:
# Записываем каждый датафрейм в соответсвующий лист
left_df = merged_df[merged_df['_merge'] == 'left_only']
left_df.drop(left_df.iloc[:,5:],axis=1,inplace=True)

In [491]:
# Очищаем от пробелов и строки названия урочища не заполнено
left_df['Урочище'] = left_df['Урочище'].apply(lambda x:x.replace('Название урочища не заполнено',''))
left_df['Урочище'] = left_df['Урочище'].apply(lambda x:x.strip())

In [492]:
for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Нет в таблице 3'].append(r)
wb['Нет в таблице 3'].column_dimensions['A'].width = 30
wb['Нет в таблице 3'].column_dimensions['B'].width = 30
    

In [493]:
# Записываем каждый датафрейм в соответсвующий лист
right_df = merged_df[merged_df['_merge'] == 'right_only']
right_df.drop(right_df.iloc[:,:6],axis=1,inplace=True)

In [494]:
# Удаляем лишние колонки
right_df.drop(right_df.iloc[:,10:],axis=1,inplace=True)

right_df.drop(columns=['ID','1','2','3','4'],axis=1,inplace=True)

In [495]:
right_df

,5,6,7,8,9
139936,Бабушкинское,Бабушкинское,,118,8
139937,Бабушкинское,Бабушкинское,,118,12
139938,Бабушкинское,Бабушкинское,,118,15
139939,Бабушкинское,Бабушкинское,,118,16
139940,Бабушкинское,Бабушкинское,,118,19
...,...,...,...,...,...
139931,Хандагатайское,Челутаевское,Название урочища не заполнено,161,14
139932,Хандагатайское,Челутаевское,Название урочища не заполнено,161,17
139933,Хандагатайское,Челутаевское,Название урочища не заполнено,173,11
139934,Хандагатайское,Челутаевское,Название урочища не заполнено,173,15


In [496]:
# Очищаем от пробелов и строки названия урочища не заполнено
right_df['7'] = right_df['7'].apply(lambda x:x.replace('Название урочища не заполнено',''))
right_df['7'] = right_df['7'].apply(lambda x:x.strip())

In [497]:
# Переименовываем колонки
right_df.columns = ['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']

In [498]:
# Записываем на лист
for r in dataframe_to_rows(right_df, index=False, header=True):
    wb['Нет в реестре УПП'].append(r)

wb['Нет в реестре УПП'].column_dimensions['A'].width = 30
wb['Нет в реестре УПП'].column_dimensions['B'].width = 30

In [499]:
# Записываем каждый датафрейм в соответсвующий лист
both_df = merged_df[merged_df['_merge'] == 'both']

In [500]:
both_df.drop(both_df.iloc[:,17:],axis=1,inplace=True)

In [501]:
both_df.head(1)

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",ID,1,2,3,4,5,6,7,8,9,10
0,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНазваниеурочищанезаполнено321,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13.0


In [502]:
both_df.drop(columns=['ID','1','2','3','4','5','6','7','8','9'],axis=1,inplace=True)

In [503]:
# Очищаем от пробелов и строки названия урочища не заполнено
both_df['Урочище'] = both_df['Урочище'].apply(lambda x:x.replace('Название урочища не заполнено',''))
both_df['Урочище'] = both_df['Урочище'].apply(lambda x:x.strip())

In [504]:
both_df.rename(columns={'Площадь лесотаксационного выдела, га':'Площадь из реестра УПП','10':'Старая площадь'},inplace=True)

In [505]:
for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Выделы с обновленной площадью'].append(r)

wb['Выделы с обновленной площадью'].column_dimensions['A'].width = 30
wb['Выделы с обновленной площадью'].column_dimensions['B'].width = 30

In [506]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
wb.save(f'{path_to_end_folder}/Данные для проверки корректности обновления площадей в таблице 3 от {current_time}.xlsx')

In [507]:
# Делаем таблицу 3
out_table3_df = merged_df[merged_df['_merge'] != 'left_only']

In [508]:
# Приводим к инту колонки чтобы отсортировать их
out_table3_df['8'] = out_table3_df['8'].apply(convert_to_int_transfer)
out_table3_df['9'] = out_table3_df['9'].apply(convert_to_int_transfer)

In [509]:
out_table3_df.sort_values(['5','6','7','8','9'],inplace=True)

In [510]:
# Присваиваем колонке 10 значение колонки созданной колонки 10
out_table3_df['10'] = out_table3_df['Площадь']

In [511]:
out_table3_df.drop(columns=['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Площадь лесотаксационного выдела, га','ID','_merge','Площадь'],inplace=True)

In [512]:
# Очищаем от пробелов и строки названия урочища не заполнено
out_table3_df['7'] = out_table3_df['7'].apply(lambda x:x.replace('Название урочища не заполнено',''))
out_table3_df['7'] = out_table3_df['7'].apply(lambda x:x.strip())


In [517]:
out_table3_df.head(3)

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,29.06.2022,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,,32,1,13.0,...,0,0,,0.0,0.0,1.0,201.0,NaN,0.0,2015.0
1,29.06.2022,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,,32,1,13.0,...,0,0,,0.0,0.0,1.0,201.0,NaN,0.0,2020.0
2,29.06.2022,NaN,0.3,Республика Бурятия,Ангоянское,Ангоянское,,32,1,13.0,...,0,0,,0.0,0.0,1.0,201.0,NaN,0.0,2021.0


In [514]:
# Приводим даты к нормальному виду ДД.ММ.ГГГГ
out_table3_df['1'] = pd.to_datetime(out_table3_df['1'],errors='coerce',dayfirst=True)
out_table3_df['22'] = pd.to_datetime(out_table3_df['22'],errors='coerce',dayfirst=True)
out_table3_df['27'] = pd.to_datetime(out_table3_df['27'],errors='coerce',dayfirst=True)
out_table3_df['1'] = out_table3_df['1'].apply(create_doc_convert_date)
out_table3_df['22'] = out_table3_df['22'].apply(create_doc_convert_date)
out_table3_df['27'] = out_table3_df['27'].apply(create_doc_convert_date)

In [515]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
itog = load_workbook(header_table3) # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(out_table3_df,index=False,header=False):
    itog['Лист1'].append(row)

In [516]:
itog.save(f'{path_to_end_folder}/Таблица 3 с присвоенными площадями из реестра УПП от {current_time}.xlsx')